## 0. Install Dependecies

In [2]:
pip install torch

Set Random seed

In [3]:
# Set random seed for reproducibility
seed = 42
random.seed(seed)  # Python random module
np.random.seed(seed)  # NumPy
torch.manual_seed(seed)  # PyTorch

# If you're using GPU, set the deterministic flag
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**Set device to GPU if is available otherwise set device as cpu**

In [4]:
import torch
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


**Import libraries**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

##1. Dataset##

- The Datatset used in this project was generated using the Mujoco simulator with three different configurations:
- 2D (2 joints)
- 2D (3 joints)
- 3D (5 joints)

The format of the data is in CSV format, including information about Joint angles, fingertip position, and orientation.

This Notebook will focus on the 2DOF Robot dataset.


1.1. Visualise data from the simulator

In [6]:
!head -5 logfiler2.csv

j0;j1;cos(j0);cos(j1);sin(j0);sin(j1);ft_x;ft_y;ft_qw;ft_qz
 0.055; -0.012;  0.998;  1.000;  0.055; -0.012;  0.210;  0.010;  1.000;  0.021
 0.076; -0.017;  0.997;  1.000;  0.076; -0.017;  0.210;  0.014;  1.000;  0.030
 0.148; -0.011;  0.989;  1.000;  0.147; -0.011;  0.208;  0.030;  0.998;  0.068
 0.214;  0.048;  0.977;  0.999;  0.212;  0.048;  0.204;  0.050;  0.991;  0.131


1.2. Preprocess the data


2R Robot

In [7]:
# Load dataset
data = pd.read_csv('logfiler2.csv', delimiter=';')

# Preprocessing: Extract inputs (joint angles and their trigonometric functions) and outputs (fingertip positions and quaternions)
X = data[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values
y = data[['ft_x' ,'ft_y' ,'ft_qw','ft_qz']].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

- Split the data into training and testing sets

In [8]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

##2. Train Forward Kinematics Models##



### 2.1. Robot 2R

- Define the architecture of the model (Feedforward Neural Network) to learn forward kinematics.




In [9]:
class ForwardKinematicsModel(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.3):
        super(ForwardKinematicsModel, self).__init__()
        # Define a feedforward network with configurable hidden_size and dropout
        self.fc1 = nn.Linear(6, hidden_size)  # Input layer
        self.dropout1 = nn.Dropout(p=dropout_rate)    # Dropout after first hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        self.dropout2 = nn.Dropout(p=dropout_rate)    # Dropout after second hidden layer
        self.fc3 = nn.Linear(hidden_size, 4)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after first hidden layer
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after second hidden layer
        x = self.fc3(x)
        return x

- Hyperparameter Search

Using Grid Search

In [11]:
# Hyperparameters
# Setting dummy values
hidden_size = 128
learning_rate = 0.001
num_epochs = 200

In [ ]:
# PyTorch Regressor Wrapper
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_size=hidden_size  , lr=learning_rate, epochs=num_epochs):
        self.hidden_size = hidden_size
        self.lr = lr
        self.epochs = epochs
        self.model = ForwardKinematicsModel(hidden_size=self.hidden_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()

    def fit(self, X, y):
        X_train = torch.tensor(X, dtype=torch.float32)
        y_train = torch.tensor(y, dtype=torch.float32)
        for epoch in range(self.epochs):
            self.model.train()
            self.optimizer.zero_grad()
            output = self.model(X_train)
            loss = self.criterion(output, y_train)
            loss.backward()
            self.optimizer.step()
        return self

    def predict(self, X):
        X_test = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X_test)
        return predictions.numpy()

# Load your dataset
df = pd.read_csv('logfiler2.csv', sep=';')
X = df[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values
y = df[['ft_x' ,'ft_y' ,'ft_qw','ft_qz']].values

# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up hyperparameters to search
param_grid = {
    'hidden_size': [32, 64, 128],  # Try different numbers of hidden units
    'lr': [0.001, 0.01],           # Try different learning rates
    'epochs': [100, 200],          # Try different numbers of epochs
}

# Instantiate the model wrapper
pytorch_model = PyTorchRegressor()

# Use GridSearchCV
grid_search = GridSearchCV(estimator=pytorch_model, param_grid=param_grid, cv=3, verbose=2, n_jobs=1)
grid_search.fit(X_train, y_train)

# Get the best two combinations of hyperparameters
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)

# Print the best and second-best hyperparameters
print("Best combination of hyperparameters:")
print(results.iloc[0][['param_hidden_size', 'param_lr', 'param_epochs', 'mean_test_score']])
print("\nSecond-best combination of hyperparameters:")
print(results.iloc[1][['param_hidden_size', 'param_lr', 'param_epochs', 'mean_test_score']])

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_loss = mean_squared_error(y_test, y_pred)
print(f"\nTest MSE of the best model: {test_loss:.4f}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=  27.9s
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=  26.4s
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=  27.6s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=  29.0s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=  31.8s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=  28.9s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=  30.8s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=  26.7s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=  28.2s
[CV] END ................epochs=100, hidden_size=64, lr=0.01; total time=  26.6s
[CV] END ................epochs=100, hidden_size=64, lr=0.01; total time=  26.5s
[CV] END ................epochs=100, hidden_size

## Train the model with the best 2 parameters found by grid search

- Train the models on joint angle inputs to predict fingertip positions.

In [ ]:
# Hyperparameters
hidden_size = 128
learning_rate = 0.001
num_epochs = 200

- Define the loss function and optimizer

In [ ]:
model = ForwardKinematicsModel(hidden_size=hidden_size)

In [ ]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Convert to PyTorch tensors only if necessary
X_train = X_train.clone().detach().float() if isinstance(X_train, torch.Tensor) else torch.tensor(X_train, dtype=torch.float32)
y_train = y_train.clone().detach().float() if isinstance(y_train, torch.Tensor) else torch.tensor(y_train, dtype=torch.float32)
X_val = X_val.clone().detach().float() if isinstance(X_val, torch.Tensor) else torch.tensor(X_val, dtype=torch.float32)
y_val = y_val.clone().detach().float() if isinstance(y_val, torch.Tensor) else torch.tensor(y_val, dtype=torch.float32)
X_test = X_test.clone().detach().float() if isinstance(X_test, torch.Tensor) else torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach().float() if isinstance(y_test, torch.Tensor) else torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Initialize variables for early stopping
best_val_loss = float('inf')
patience = 4
no_improvement_epochs = 0
best_model_path = "best_model.pth"  # Path to save the best model

# Training loop with early stopping
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate on validation set
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)

            # Calculate additional metrics
            mae = mean_absolute_error(y_val.numpy(), val_outputs.numpy())
            r2 = r2_score(y_val.numpy(), val_outputs.numpy())

            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"MAE: {mae:.4f}, R^2: {r2:.4f}")

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improvement_epochs = 0
                # Save the best model
                torch.save(model.state_dict(), best_model_path)
                print(f"Model saved at epoch {epoch+1} with Val Loss: {val_loss:.4f}")
            else:
                no_improvement_epochs += 1

            if no_improvement_epochs >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")

        model.train()  # Switch back to training mode

In [ ]:
# Initialize variables for early stopping and visualization
best_val_loss = float('inf')
patience = 4
no_improvement_epochs = 0
best_model_path = "best_model.pth"  # Path to save the best model

# Lists to store metrics for visualization
train_losses = []
val_losses = []
val_maes = []
val_r2_scores = []

# Training loop with early stopping
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Store train loss for visualization
    train_losses.append(loss.item())

    # Evaluate on validation set
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss.item())  # Store validation loss

            # Calculate additional metrics
            mae = mean_absolute_error(y_val.numpy(), val_outputs.numpy())
            r2 = r2_score(y_val.numpy(), val_outputs.numpy())
            val_maes.append(mae)  # Store validation MAE
            val_r2_scores.append(r2)  # Store validation R^2

            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"MAE: {mae:.4f}, R^2: {r2:.4f}")

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improvement_epochs = 0
                # Save the best model
                torch.save(model.state_dict(), best_model_path)
                print(f"Model saved at epoch {epoch+1} with Val Loss: {val_loss:.4f}")
            else:
                no_improvement_epochs += 1

            if no_improvement_epochs >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")
                break  # Exit the training loop early

        model.train()  # Switch back to training mode

In [ ]:
# Visualization
epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(15, 5))

# Plot training and validation loss
plt.subplot(1, 3, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(range(10, len(val_losses)*10 + 1, 10), val_losses, label="Validation Loss", linestyle="--")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()

# Plot validation MAE
plt.subplot(1, 3, 2)
plt.plot(range(10, len(val_maes)*10 + 1, 10), val_maes, label="Validation MAE", color="orange")
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.title("Validation Mean Absolute Error")
plt.legend()

# Plot validation R^2
plt.subplot(1, 3, 3)
plt.plot(range(10, len(val_r2_scores)*10 + 1, 10), val_r2_scores, label="Validation R^2", color="green")
plt.xlabel("Epochs")
plt.ylabel("R^2 Score")
plt.title("Validation R^2 Score")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    mae_test = mean_absolute_error(y_test.numpy(), test_outputs.numpy())
    r2_test = r2_score(y_test.numpy(), test_outputs.numpy())

    print(f"Test Loss: {test_loss.item():.4f}")
    print(f"Test MAE: {mae_test:.4f}")
    print(f"Test R^2: {r2_test:.4f}")


##3. Compare Jacobians##



3.1. Compute the Jacobian matrix for the learned forward kinematics using automatic differentiation.



In [ ]:
X_test = X_test.clone().detach().float() if isinstance(X_test, torch.Tensor) else torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach().float() if isinstance(y_test, torch.Tensor) else torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Assuming X_test is a tensor and you want to extract the first sample
X_test_sample = X_test[0].clone().detach().unsqueeze(0).float()  # Add batch dimension and detach
print(X_test_sample)

In [ ]:
# Load the best model after training
model.load_state_dict(torch.load(best_model_path))
print("Best model loaded for further evaluation or inference.")

In [ ]:
def compute_2x2_jacobian(model, inputs):
    # Ensure input gradients are enabled
    if not inputs.requires_grad:
        inputs.requires_grad = True

    # Forward pass
    outputs = model(inputs)

    # Select outputs of interest
    ft_x, ft_y = outputs[2], outputs[3]

    # Initialize the 2x2 Jacobian matrix
    jacobian = torch.zeros(2, 2)

    # Compute gradients for ft_x
    grad_ft_x = torch.autograd.grad(ft_x, inputs, retain_graph=True, create_graph=True)[0]
    jacobian[0, 0] = grad_ft_x[0]  # ∂ft_x/∂j0
    jacobian[0, 1] = grad_ft_x[1]  # ∂ft_x/∂j1

    # Compute gradients for ft_y
    grad_ft_y = torch.autograd.grad(ft_y, inputs, retain_graph=True, create_graph=True)[0]
    jacobian[1, 0] = grad_ft_y[0]  # ∂ft_y/∂j0
    jacobian[1, 1] = grad_ft_y[1]  # ∂ft_y/∂j1

    return jacobian

3.2. Compare the computed Jacobian with the analytical Jacobian for the 2-joint robot.

In [ ]:
import numpy as np

# Analytical Jacobian for a 2-DOF robot
def analytical_jacobian_from_test(data):
    # Extract joint angles from data
    j0 = data[0]  # j0
    j1 = data[1]  # j1

    l1 = 1.0  # Link 1 length
    l2 = 1.0  # Link 2 length

    # Calculate the Jacobian elements using the analytical formula
    J = np.array([
        [-l1 * np.sin(j0) - l2 * np.sin(j0 + j1), -l2 * np.sin(j0 + j1)],
        [l1 * np.cos(j0) + l2 * np.cos(j0 + j1), l2 * np.cos(j0 + j1)]
    ])
    return J

- Compute Diference between jacobian one iteration

In [ ]:
import numpy as np
import torch

#Numerical Jacobian (calculated by FK_Jacobian)
X_test_sample = torch.tensor([-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930], requires_grad=True)
jacobian_numerical = compute_2x2_jacobian(model, X_test_sample)

# Analytical Jacobian (calculated using analytical formula)X_test_sample = [-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930] # Extract joint angles for comparison
X_test_sample = X_test_sample.detach().numpy()
J_analytical = analytical_jacobian_from_test(X_test_sample)

# Print both Jacobians
print("Numerical Jacobian (PyTorch):\n", jacobian_numerical)
print("Analytical Jacobian (2-DOF Robot):\n", J_analytical)

# Compare both Jacobians by calculating the difference
jacobian_difference = (jacobian_numerical.detach().numpy() - J_analytical)
print("Difference between Numerical and Analytical Jacobians:\n", jacobian_difference)

- Compute diferences Full test set

In [ ]:
# Initialize lists to store results
numerical_jacobians = []
analytical_jacobians = []
jacobian_differences = []

# Loop through the test set to compute the Jacobians and differences
for i in range(len(X_test)):
    # Get the current sample (convert to tensor and ensure requires_grad=True)
    X_test_sample = torch.tensor(X_test[i], dtype=torch.float32, requires_grad=True)

    # Compute the numerical Jacobian using the model
    jacobian_numerical = compute_2x2_jacobian(model, X_test_sample)

    # Compute the analytical Jacobian
    X_test_sample_np = X_test_sample.detach().numpy()
    J_analytical = analytical_jacobian_from_test(X_test_sample_np)

    # Calculate the difference between the Jacobians
    jacobian_difference = jacobian_numerical.detach().numpy() - J_analytical

    # Store the results
    numerical_jacobians.append(jacobian_numerical.detach().numpy())
    analytical_jacobians.append(J_analytical)
    jacobian_differences.append(jacobian_difference)

# Convert lists to numpy arrays for analysis
numerical_jacobians = np.array(numerical_jacobians)
analytical_jacobians = np.array(analytical_jacobians)
jacobian_differences = np.array(jacobian_differences)

# Calculate the mean difference
mean_difference = np.mean(np.abs(jacobian_differences), axis=(1, 2))  # Mean of absolute differences for each sample
mean_difference_overall = np.mean(mean_difference)  # Overall mean difference

# Calculate percentage of times analytical Jacobian is bigger or smaller
analytical_bigger = np.sum(np.linalg.norm(analytical_jacobians, axis=(1, 2)) > np.linalg.norm(numerical_jacobians, axis=(1, 2)))
analytical_smaller = np.sum(np.linalg.norm(analytical_jacobians, axis=(1, 2)) < np.linalg.norm(numerical_jacobians, axis=(1, 2)))
total = len(X_test)

percent_bigger = (analytical_bigger / total) * 100
percent_smaller = (analytical_smaller / total) * 100

# Print results
print(f"Mean difference between numerical and analytical Jacobians: {mean_difference_overall:.6f}")
print(f"Percentage of cases where analytical Jacobian is bigger: {percent_bigger:.2f}%")
print(f"Percentage of cases where analytical Jacobian is smaller: {percent_smaller:.2f}%")

<ipython-input-69-fe2229e38e07>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_sample = torch.tensor(X_test[i], dtype=torch.float32, requires_grad=True)


A saída de streaming foi truncada nas últimas 5000 linhas.
ft_y: tensor(0.8479, grad_fn=<SelectBackward0>)
ft_x: tensor(-0.4301, grad_fn=<SelectBackward0>)
ft_y: tensor(0.7723, grad_fn=<SelectBackward0>)
ft_x: tensor(-0.9064, grad_fn=<SelectBackward0>)
ft_y: tensor(-0.3455, grad_fn=<SelectBackward0>)
ft_x: tensor(-0.1314, grad_fn=<SelectBackward0>)
ft_y: tensor(-0.8526, grad_fn=<SelectBackward0>)
ft_x: tensor(0.8842, grad_fn=<SelectBackward0>)
ft_y: tensor(-0.3390, grad_fn=<SelectBackward0>)
ft_x: tensor(-0.9753, grad_fn=<SelectBackward0>)
ft_y: tensor(-0.3227, grad_fn=<SelectBackward0>)
ft_x: tensor(0.1987, grad_fn=<SelectBackward0>)
ft_y: tensor(0.8671, grad_fn=<SelectBackward0>)
ft_x: tensor(0.9417, grad_fn=<SelectBackward0>)
ft_y: tensor(-0.0018, grad_fn=<SelectBackward0>)
ft_x: tensor(-0.9549, grad_fn=<SelectBackward0>)
ft_y: tensor(0.3672, grad_fn=<SelectBackward0>)
ft_x: tensor(0.8956, grad_fn=<SelectBackward0>)
ft_y: tensor(-0.0165, grad_fn=<SelectBackward0>)
ft_x: tensor(-0.62

KeyboardInterrupt: 

## Train the model now with different Hyperparameters

- second best according to grid search

In [ ]:
# Hyperparameters
hidden_size = 128
learning_rate = 0.001
num_epochs = 200

In [ ]:
model = ForwardKinematicsModel(hidden_size=hidden_size)

In [ ]:
# Initialize variables for early stopping
best_val_loss = float('inf')
patience = 4
no_improvement_epochs = 0
best_model_path = "best_model_2.pth"  # Path to save the best model

# Training loop with early stopping
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate on validation set
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)

            # Calculate additional metrics
            mae = mean_absolute_error(y_val.numpy(), val_outputs.numpy())
            r2 = r2_score(y_val.numpy(), val_outputs.numpy())

            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"MAE: {mae:.4f}, R^2: {r2:.4f}")

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improvement_epochs = 0
                # Save the best model
                torch.save(model.state_dict(), best_model_path)
                print(f"Model saved at epoch {epoch+1} with Val Loss: {val_loss:.4f}")
            else:
                no_improvement_epochs += 1

            if no_improvement_epochs >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")

        model.train()  # Switch back to training mode

In [ ]:
# Visualization
epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(15, 5))

# Plot training and validation loss
plt.subplot(1, 3, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(range(10, len(val_losses)*10 + 1, 10), val_losses, label="Validation Loss", linestyle="--")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()

# Plot validation MAE
plt.subplot(1, 3, 2)
plt.plot(range(10, len(val_maes)*10 + 1, 10), val_maes, label="Validation MAE", color="orange")
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.title("Validation Mean Absolute Error")
plt.legend()

# Plot validation R^2
plt.subplot(1, 3, 3)
plt.plot(range(10, len(val_r2_scores)*10 + 1, 10), val_r2_scores, label="Validation R^2", color="green")
plt.xlabel("Epochs")
plt.ylabel("R^2 Score")
plt.title("Validation R^2 Score")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    mae_test = mean_absolute_error(y_test.numpy(), test_outputs.numpy())
    r2_test = r2_score(y_test.numpy(), test_outputs.numpy())

    print(f"Test Loss: {test_loss.item():.4f}")
    print(f"Test MAE: {mae_test:.4f}")
    print(f"Test R^2: {r2_test:.4f}")

In [ ]:
X_test = X_test.clone().detach().float() if isinstance(X_test, torch.Tensor) else torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach().float() if isinstance(y_test, torch.Tensor) else torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Assuming X_test is a tensor and you want to extract the first sample
X_test_sample = X_test[0].clone().detach().unsqueeze(0).float()  # Add batch dimension and detach
print(X_test_sample)

In [ ]:
import numpy as np
import torch

#Numerical Jacobian (calculated by FK_Jacobian)
X_test_sample = torch.tensor([-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930], requires_grad=True)
jacobian_numerical = compute_2x2_jacobian(model, X_test_sample)

# Analytical Jacobian (calculated using analytical formula)X_test_sample = [-3.0970, -2.6260, -0.9990, -0.8700, -0.0450, -0.4930] # Extract joint angles for comparison
X_test_sample = X_test_sample.detach().numpy()
J_analytical = analytical_jacobian_from_test(X_test_sample)

# Print both Jacobians
print("Numerical Jacobian (PyTorch):\n", jacobian_numerical)
print("Analytical Jacobian (2-DOF Robot):\n", J_analytical)

# Compare both Jacobians by calculating the difference
jacobian_difference = (jacobian_numerical.detach().numpy() - J_analytical)
print("Difference between Numerical and Analytical Jacobians:\n", jacobian_difference)

In [ ]:
# Initialize lists to store results
numerical_jacobians = []
analytical_jacobians = []
jacobian_differences = []

# Loop through the test set to compute the Jacobians and differences
for i in range(len(X_test)):
    # Get the current sample (convert to tensor and ensure requires_grad=True)
    X_test_sample = torch.tensor(X_test[i], dtype=torch.float32, requires_grad=True)

    # Compute the numerical Jacobian using the model
    jacobian_numerical = compute_2x2_jacobian(model, X_test_sample)

    # Compute the analytical Jacobian
    X_test_sample_np = X_test_sample.detach().numpy()
    J_analytical = analytical_jacobian_from_test(X_test_sample_np)

    # Calculate the difference between the Jacobians
    jacobian_difference = jacobian_numerical.detach().numpy() - J_analytical

    # Store the results
    numerical_jacobians.append(jacobian_numerical.detach().numpy())
    analytical_jacobians.append(J_analytical)
    jacobian_differences.append(jacobian_difference)

# Convert lists to numpy arrays for analysis
numerical_jacobians = np.array(numerical_jacobians)
analytical_jacobians = np.array(analytical_jacobians)
jacobian_differences = np.array(jacobian_differences)

# Calculate the mean difference
mean_difference = np.mean(np.abs(jacobian_differences), axis=(1, 2))  # Mean of absolute differences for each sample
mean_difference_overall = np.mean(mean_difference)  # Overall mean difference

# Calculate percentage of times analytical Jacobian is bigger or smaller
analytical_bigger = np.sum(np.linalg.norm(analytical_jacobians, axis=(1, 2)) > np.linalg.norm(numerical_jacobians, axis=(1, 2)))
analytical_smaller = np.sum(np.linalg.norm(analytical_jacobians, axis=(1, 2)) < np.linalg.norm(numerical_jacobians, axis=(1, 2)))
total = len(X_test)

percent_bigger = (analytical_bigger / total) * 100
percent_smaller = (analytical_smaller / total) * 100

# Print results
print(f"Mean difference between numerical and analytical Jacobians: {mean_difference_overall:.6f}")
print(f"Percentage of cases where analytical Jacobian is bigger: {percent_bigger:.2f}%")
print(f"Percentage of cases where analytical Jacobian is smaller: {percent_smaller:.2f}%")